In [13]:
!pip install python-dotenv
!pip install odp-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 491.1 kB/s  0:00:14m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 24.9/29.6 MB 524.7 kB/s eta 0:00:09
Resuming download pyarrow-18.1.0-cp311-cp311-macosx_12_0_arm64.whl (24.9 MB/29.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 6.4 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: validators
    Found existing installation: validators 0.35.0
    Uninstalling validators-0.35.0:
      Successfully uninstalled validators-0.35.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.0.0
    Uninstalling pyarrow-18.0.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/10 [pyarrow]
      Successfully uninstalled pyarrow-18.0.0━━━━━━━━━━━━━━━━━  3/10 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [odp-sdk]9/10 [odp-sdk]aphy]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [ ]:
import os 
api_key = os.environ.get('ODP_API_KEY')
print(api_key != None)

False
True


In [ ]:
#first test the api key with the sample from https://app.hubocean.earth/account

from odp.client import OdpClient
client = OdpClient(api_key=api_key)
dataset_name = "42cae68c-245a-4489-af48-6ad841b9ef65"
dataset = client.catalog.get(f"catalog.hubocean.io/dataset/{dataset_name}")
print(dataset)
tab = client.table_v2(dataset)
for row in tab.select().rows():
  print(row)
#for the moment this returns HTTP Error - 503: no available server

#ignore this and try features catalog...


/var/folders/4c/mxsj6f9d5_x8cjryn16fb1c40000gn/T/ipykernel_79839/3393960434.py:4: DeprecationWarning: catalog class is deprecated and will be removed in a future versions. Use catalog_v2 class instead.
  dataset = client.catalog.get(f"catalog.hubocean.io/dataset/{dataset_name}")


HTTPError: HTTP Error - 503: no available server


In [ ]:
#I've already used QGIS for this connection to preview some data, but qgis does not allow to discover all the option available.ValueError

# I'll use OWSlib 
import owslib
from owslib.ogcapi.features import Features
# connect to OGC Features endpoint with OWSLIB
_headers = {"Authorization": f"ApiKey sk_26baf4fa04be54136b2aa34b"}
odp_features_url = "https://api.hubocean.earth/api/features"
w = Features(odp_features_url, headers=_headers)

conformance = w.conformance()
conformance

{'conformsTo': ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/html']}

So we know Features part 1 is implemented with this response
{'conformsTo': ['http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/html']}

it hould provide openAPI descritpion, lets check

In [31]:
w.api()

{'openapi': '3.0.3',
 'info': {'title': 'Ocean Data Platform OGC API – Features',
  'version': '0.1.0',
  'description': 'OGC API Features core.\n\n'},
 'servers': [{'url': 'https://api.hubocean.earth/api/features',
   'description': 'Default server'}],
 'paths': {'/': {'get': {'summary': 'Landing page',
    'responses': {'200': {'description': 'Landing page'}}}},
  '/conformance': {'get': {'summary': 'Conformance declaration',
    'responses': {'200': {'description': 'Conformance set'}}}},
  '/collections': {'get': {'summary': 'List collections',
    'responses': {'200': {'description': 'Collections list'}}}},
  '/collections/{collectionId}': {'get': {'summary': 'Describe collection',
    'parameters': [{'name': 'collectionId',
      'in': 'path',
      'required': True,
      'schema': {'type': 'string'}}],
    'responses': {'200': {'description': 'Collection metadata'}}}},
  '/collections/{collectionId}/items': {'get': {'summary': 'Fetch features',
    'parameters': [{'name': 'colle

Quicly looking looks useful. We know how can we query on the collections level
 - maybe /openapi itself could be added 
 

quicly looking looks usefule